### TEST DU CODE

In [17]:
import chess.svg
from anytree import NodeMixin
import numpy as np
import chess
from random import sample
from IPython.display import display
from MCTS import *
from MCTS_nn import *

In [18]:
game = chess.Board()
MCTS = mcts_nn(game)

while game.outcome() is None:

    for i in range(100):
        leaf = MCTS.selection()
        child_node, result = MCTS.expansion(leaf)
        MCTS.backpropagation(child_node,result)
    
    N = [child.N for child in MCTS.root.children]
    print([[child.N, child.move] for child in MCTS.root.children])
    print(sum([child.N for child in MCTS.root.children]))
    index = sample([i for i, j in enumerate(N) if j == max(N)],1)[0] 
    move = [child.move for child in MCTS.root.children][index]
    game.push(move)

    # tree reuse
    MCTS.root = [child for child in MCTS.root.children][index]
    MCTS.root.parent = None
    MCTS.initial_position = game.copy()
    MCTS.current_position = game.copy()
    #MCTS = mcts(game)

    display(game)

'''CODER LA PARTIE CONTRE HUMAIN'''
display(game)  
    

FileNotFoundError: [Errno 2] No such file or directory: '/content/Reinforcement-Learning-AlphaZero/codes/model_config.json'

### NOUVELLE CLASSE MCTS NN

In [ ]:
class node_nn(NodeMixin):

    def __init__(self,move=None,parent=None,prob=0,V=0):
        self.move = move 
        self.N = 0
        self.V = V
        self.prob = prob
        self.parent = parent

    def score(self,white_to_play):
        if white_to_play:
            relative_W = self.V
        else:
            relative_W = -self.V
        return relative_W/(self.N or 1) + self.prob * np.sqrt(self.parent.N) / (1 + self.N)
        


class mcts_nn():
    def __init__(self,position):
        self.initial_position = position.copy() # initial_position and current_position are chess.Board() objects 
        self.current_position = position.copy()
        self.root = node_nn() # create the tree root, that correspond to initial_position

    def selection(self): # we reach a leaf using score selection
        current_node = self.root # we start from the root
        while current_node.is_leaf is not True: # check if we are in a leaf
            white_to_play = self.current_position.turn
            score = [child.score(white_to_play) for child in current_node.children] # list of scores for the node's children
            index = sample([i for i, j in enumerate(score) if j == max(score)],1)[0] # si plusieurs max tirage au hasard
            current_node = current_node.children[index] # the best children (with maximal UCT) becomes the current node
            self.current_position.push(current_node.move) # mise à jour de la position courante pour la phase d'expansion / simulation
        leaf = current_node # the current node is a leaf by definition
        return leaf

    def expansion_simulation(self,leaf): 
        outcome = self.current_position.outcome() 
        #EXPANSION
        legal_moves = self.current_position.legal_moves # génération des coups légaux
        child_node = None # juste pour ne pas bugger la fin de partie

        if outcome is None: # si la partie n'est pas terminée
            # pour chaque coup jouable, on évalue la position à l'aide du nn
            # on stocke p dans les enfants et on fait remonter v dans les ancètres
            p,v = evaluate_position(model, position)
            for move in legal_moves: # création des nouveaux noeuds correspondants aux coups légaux

                node_nn(move=move,parent=leaf,)
            self.current_position.push(chosen_move) # mise à jour de la position courante avec le nouveau coup
            outcome = self.current_position.outcome() # mise à jour de l'issue de la partie
        #SIMULATION
        while outcome is None : # tant que la partie n'est pas terminée
            legal_moves = self.current_position.legal_moves # génération des coups légaux
            random_move = sample(list(legal_moves),1)[0] # tirage aléatoire du prochain coup
            self.current_position.push(random_move) # mise à jour de la position courante avec le nouveau coup
            outcome = self.current_position.outcome() # mise à jour de l'issue de la partie
        result = outcome.result() 

        return child_node, result

    def backpropagation(self, child_node, result): # we backpropagate information through the tree
        if result == "1-0":
            inc = 1
        elif result == "0-1":
            inc = -1
        else:
            inc = 0
        if child_node is not None :  
            for ancestor in child_node.iter_path_reverse(): # rétropropagation du résultat issu de la simulation
                ancestor.N += 1
                ancestor.W += inc
        self.current_position = self.initial_position.copy()
        return

In [6]:
pos = chess.Board()

In [7]:
from utils import *

In [8]:
moves = create_uci_labels()

In [12]:
pos.push(chess.Move.from_uci('e2e4'))

In [16]:
[chess.Move.from_uci(move) in pos.legal_moves for move in moves]

20

In [21]:
pos.outcome()